# Brief introduction to TDD Test Types - In Python! 

This document will have some code and exlanations of the following:

* Dummy
* Fake
* Stub
* Mock
* Spy

For these examples we will be using the _unittest_ library in python, but most concepts apply to other languages or libraries. 

##### Disclaimer: I mainly made this for self understanding. Any comment, correction, clarification or sugestion is well received, just add an issue over the repo.

In [1]:
import unittest

Now Let's say we are building an Insect identification system. The unit that we want to test consists of two methods, one to discern arachnids from insects, and the other one for retrieving the insect type, which calls the _isArachnid_ function  

In [2]:
def isArachnid(legs):
    if legs ==8:
        return True
    else:
        return False

def insectType(legs, color):
    if (isArachnid(legs)):
        return "Arachnid"
    else:
        if color == "black":
            return "Cockroach"
        else:
            return "Other insect"
    # ...

## Dummy

A Dummy is a placeholder required to pass the unit test. Unit in the context (SUT) doesn’t exercise this placeholder. A dummy can be something as simple as passing ‘null’ or a void implementation with exceptions to ensure it’s never leveraged. Dummies should always try to be explicit so it doesn't confuse the developer reading the tests, into thinking that the dummy value holds additional meaning. Defaults and empty values should be used.

Let's say we write a test to check whether we can identify arachnids from other insects. Siince the _color_ attribute is not needed, we will send an empty string instead. This is what we call a dummy. Dummies could also be more complex objects, as long as proper constructors are provided.

In [3]:
class InsectTests(unittest.TestCase):

    def test_it_identifies_arachnids(self):
        arachnid_legs= 8
        dummy_arachnid_color = str() # <---  empty string as a dummy value
        insect_type = insectType(arachnid_legs, dummy_arachnid_color)
        self.assertEqual(insect_type, "Arachnid")


And we run the tests:....

In [4]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


## Fake

A Fake is used to simplify a dependecy, in order to make our tests pass easily. Fakes usually are the working implementation of an interface, but they are not prduction ready, most of the time they will return just pre-defined static values. Fakes are frequently used to mimic database functionalities using in-memory databases or pre-defined files.

In this case, let's pretend we have a _ForestAPI_ that returns insects in real time that are spotted by cameras inside a forest. APIs usually have a limited quota, so we will write a fake object that mimics the values that the real API would return.

Our Fake API will  return a random insect fro a pre-defined list when we call the __getInsect()__ function.

In [5]:
import random
class ForestAPI_fake():
    def __init__(self):
        #fake insect list:

        self.insects = [
            {
                "name": "Cockroach",
                "legs": 6,
                "color": "black"
            },
            {
                "name": "Spider",
                "legs":8,
                "color": "green"
            },
            {
                "name": "Ladybug",
                "legs": 6,
                "color": "red"
            }]
    def getInsect(self):
        #We will return a random insect:
        return self.insects[random.randint(0,2)]
    def acknowledgeRelease(self):
        return True

We can check that running the fake api, we will be returned with a simple object from our fake list.

We expect the real APi to return more diverse data, and not do it in a random manner. 

In [6]:
api = ForestAPI_fake()
api.getInsect()

{'color': 'red', 'legs': 6, 'name': 'Ladybug'}

Now, lets re-write our tests and add a method to check wether our calls to the API work or not.

In [7]:
class InsectTests(unittest.TestCase):

    def test_it_identifies_arachnids(self):
        arachnid_legs= 8
        dummy_arachnid_color = str() # <---  empty string as a dummy value
        insect_type = insectType(arachnid_legs, dummy_arachnid_color)
        self.assertEqual(insect_type, "Arachnid")
        
    def test_it_retrieves_insects_from_api(self):
        api = ForestAPI_fake()
        insect = api.getInsect()
        self.assertIsNotNone(insect) # For now, we will just check that the object returned contains something.

In [8]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


There are many libraries to create "Fake" objects from API calls. These are very useful to speed up your tests since they save the API response inside a file. The next time you call that API method, instead of calling the actual API over the internet, you are returned the stored response from your filesystem. You can also choose to re-create the _cassetes_ if the API changes, or you want to have more updated data for your tests.

For python, there is [vcr.py](https://github.com/kevin1024/vcrpy), as well as vcr for [ruby](https://github.com/vcr/vcr) and [PHP](https://github.com/php-vcr/php-vcr). Just search for _"vcr + language"_ to check if there is a similar implementation for that language.

## Stub

A Stub is an object that holds predefined data and uses it to answer calls during tests. It is used when we cannot or don’t want to involve objects that would answer with real data or have undesirable side effects. 

Martin Fowler defines stubs as objects that "provide canned answers to calls made during the test, usually not responding at all to anything outside what's programmed in for the test".

The main difference with fakes, is that a stub doesn't need to be a full implementation, as long as it returns the expected values and receive the expected parameters. 
Also stubs are often automatically created by external libraries. Because Python lacks the concept of interfaces, and instead uses duck-typing, they way of doing stubbing is less standard than in other languages like Java. 

We will use the included _mock_ library in Python 3.0

In [9]:
import unittest.mock
from unittest.mock import MagicMock

Our ForestAPI will ocasionally return an insect with a legcount of 0. We have to able to identify them to remove them from further processing. We will add a method to identify if an insect is dead:

In [10]:
def isInsectDead(insect):
    legcount =  insect['legs']
    if (legcount > 0):
        return False
    else:
        return True

We will add a new test case to see if it works properly.

We cannot use the fake _ForestAPI_ to test this, because we don't have any value for a dead insect. Instead of adding one, we will define a use a stub that returns a dead insect at all times me for this test:

In [11]:
class InsectTests(unittest.TestCase):

    def test_it_identifies_arachnids(self):
        arachnid_legs= 8
        dummy_arachnid_color = str() # <---  empty string as a dummy value
        insect_type = insectType(arachnid_legs, dummy_arachnid_color)
        self.assertEqual(insect_type, "Arachnid")
        
    def test_it_retrieves_insects_from_api(self):
        api = ForestAPI_fake()
        insect = api.getInsect()
        self.assertIsNotNone(insect) # For now, we will just check that the object returned contains something.
        
    def test_it_identifies_dead_insects(self):
        api = ForestAPI_fake() 
        api.getInsect =  MagicMock(return_value= {"name": "dead ant", "legs": 0, "color": "black"}) #<- Our dead insect stub
        insect = api.getInsect()
        self.assertTrue(isInsectDead(insect)) # For now, we will just check that the object returned contains something.
        

We can see the tests passing. If we comment the stub line, we can see that the test will fail every time. And t would also fail sometimes even if we had a dead insect to the fake API.

In [12]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.007s

OK


## Mocks ##

Mocks are similar to stubs, but they aso specify behavior in them.
Martin Fowler defines mocks as: _"objects pre-programmed with expectations which form a specification of the calls they are expected to receive."_.

While with stubs we test states, mocks are intended to test behaviour.

### Stub vs Mock

##### Stub:
* Setup
* Test
* Verify State
* Teardown

##### Mock
* Setup
* __Setup Expectations__
* Test
* __Verify Expectations__
* Verify State
* Teardown

To test with mocks this, let's make an "Insect Lab" class, which contains some of the methods described before. We will also have a method to releasean insect if it is an arachnid. (I know... the logic is very stupid, but the redundant calls will show how the mocks can record that information)

The basic idea, is that we will get one insect from our _ForestAPI_ , then check whether it is an Arachnid or not to release it.

In [13]:
class InsectLab(object):
    def shouldReleaseInsect(self,insect):
        itype =  self.insectType(insect)
        if (itype == "Arachnid"):
            
            return True
        else:
           
            return False            
        
    def isArachnid(self,legs):
        if legs ==8:
            return True
        else:
            return False

    def insectType(self,insect):
        if (self.isArachnid(insect['legs'])):
            return "Arachnid"
        else:
            if insect['color'] == "black":
                return "Grounder"
            else:
                return "Other insect"
        # ...
    def getAndProcessInsect(self,api):
        insect =  api.getInsect()
        was_released = self.shouldReleaseInsect(insect)
        if (was_released):
            api.acknowledgeRelease()
        return was_released


    

This time lets make a test that fails. The mock will be the API (We built it over the fake api used before using the _(spec=...)_ command.


In our test, we will check if our mock api calls the __AcknowledgeRelease()__ function. If it is called, it means that the whole process from getting an insect, analyzing it and then releasing the arachnids works. 

In [14]:
from unittest.mock import Mock
class InsectTests(unittest.TestCase):

    def test_it_identifies_arachnids(self):
        arachnid_legs= 8
        dummy_arachnid_color = str() # <---  empty string as a dummy value
        insect_type = insectType(arachnid_legs, dummy_arachnid_color)
        self.assertEqual(insect_type, "Arachnid")
        
    def test_it_retrieves_insects_from_api(self):
        api = ForestAPI_fake()
        insect = api.getInsect()
        self.assertIsNotNone(insect) # For now, we will just check that the object returned contains something.
        
    def test_it_identifies_dead_insects(self):
        api = ForestAPI_fake() 
        api.getInsect =  MagicMock(return_value= {"name": "dead ant", "legs": 0, "color": "black"}) #<- Our dead insect stub
        insect = api.getInsect()
        self.assertTrue(isInsectDead(insect)) # For now, we will just check that the object returned contains something.
        
    def test_it_releases_arachnids(self):
        mock_api =  Mock(spec=ForestAPI_fake) 
        mock_api.getInsect.return_value = {"name": "Ant", "legs": 6, "color": "black"} #<- Our non arachnid stub
        mock_api.acknowledgeRelease = MagicMock(return_value = True)
        lab = InsectLab()
        
        lab.getAndProcessInsect(mock_api)        
        mock_api.acknowledgeRelease.assert_called_with() #<- We want to check if the release method was called

        
unittest.main(argv=['first-arg-is-ignored'], exit=False)     

..F.
FAIL: test_it_releases_arachnids (__main__.InsectTests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-14-55e0b60ed041>", line 28, in test_it_releases_arachnids
    mock_api.acknowledgeRelease.assert_called_with() #<- We want to check if the release method was called
  File "C:\Users\isaac\Anaconda3\lib\unittest\mock.py", line 783, in assert_called_with
    raise AssertionError('Expected call: %s\nNot called' % (expected,))
AssertionError: Expected call: acknowledgeRelease()
Not called

----------------------------------------------------------------------
Ran 4 tests in 0.015s

FAILED (failures=1)


We can observe that the method was not called. If we switch the insect stub to a __"legs":8__, the test passes, even tho we didn't call the real api, or any of their methods.

With mocks, we can check if a method is called, how many times it is called and more things that with stubs.

## Spies

According to Martin Fowler "Spies are stubs that also record some information based on how they were called. One form of this might be an email service that records how many messages it was sent.".

Spies should act as normal objects, with the added mock functionalities. 

On this specific python library, we can create a Mock with the parameter __(wraps=...)__ where the wrapped object will be the one we spy upon.



In [24]:
import unittest.mock  as mock

lab = InsectLab()
mock_api =  ForestAPI_fake()
spy_api = mock.Mock(wraps=mock_api)

for i in range(3):
    lab.getAndProcessInsect(spy_api)
     
    print ("Calls: %s \n" % spy_api.mock_calls)


Calls: [call.getInsect(), call.acknowledgeRelease()] 

Calls: [call.getInsect(), call.acknowledgeRelease(), call.getInsect()] 

Calls: [call.getInsect(),
 call.acknowledgeRelease(),
 call.getInsect(),
 call.getInsect()] 



We ran the  _getAndProcessInsect()_ function three times. We can see the historical calls made by the fake _ForestAPI_. In some cases, the _acknowledgeRelease()_ was called, which means the insect retrieved was an arachnid.

With Spies we can check how many times a method is being called, what are the values of the parameters it was called with, and more. Check the documentation for the specific mocking library that you use.


Happy testing!

## Sources:

* https://martinfowler.com/articles/mocksArentStubs.html
* http://www.telerik.com/blogs/fakes-stubs-and-mocks
* https://nirajrules.wordpress.com/2011/08/27/dummy-vs-stub-vs-spy-vs-fake-vs-mock/
* http://igorsobreira.com/2010/12/15/mocks-stubs-fakes-spies.html
* https://dev.to/milipski/test-doubles---fakes-mocks-and-stubs
* https://semaphoreci.com/community/tutorials/getting-started-with-mocking-in-python
* https://docs.python.org/3/library/unittest.mock-examples.html
* http://blog.thedigitalcatonline.com/blog/2016/03/06/python-mocks-a-gentle-introduction-part-1/#.WdwJVmhSyUl


...And of course, StackOverflow.
